# Input data requirements

The input ice sheet model should be a netCDF file. 


### `Lithk` variable
The uploaded model to contain thickness data (the `lithk` variable) for the comparison.


In [1]:
import os,sys
import glob

# Navigate two levels up to reach main CmCt dir
cmct_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))

# Import utilities for this comparison
sys.path.insert(0, os.path.join(cmct_dir, 'bin'))
from CmCtpackage.imbie import *

# Suppress numpy.dtype size changed warnings
import warnings
warnings.filterwarnings('ignore')

# Configure IMBIE comparison

In [2]:
# Flag for the ice sheet region Greenland or Antarctica
icesheet = "AIS"# Change to "AIS" or "GIS"


# Start and end dates for comparison
start_date = '2007-01-01'
end_date ='2014-01-31'

# Density of ice used in the model
rho_ice = 918 # (kg/m^3)

# Output file directory
output_path = cmct_dir + '/notebooks/IMBIE/'

# Select IMBIE variable for mass change comparision (name of column in IMBIE dataset)
mass_balance_column="Cumulative mass balance (Gt)"
if mass_balance_column == "Cumulative mass balance (Gt)":
    mass_balance_type = "total"
elif mass_balance_column == "Cumulative dynamics mass balance anomaly (Gt)":
    mass_balance_type = "dynamic"

# Set template model filename, shapefile for basin partitioning, projection, and IMBIE dataset filename
if icesheet == "GIS":
    projection = "EPSG:3413"  
    
    # Template for the model filenames
    mod_filename_template = '/home/jovyan/shared-public/CmCt/models/ISMIP6/lithk_GIS_*_*_historical.nc'
    
    # Shapefile for basin partitioning
    shape_filename = cmct_dir + '/data/IMBIE/Greenland_Basins_PS_v1.4.2/Greenland_Basins_PS_v1.4.2.shp'
    
    # IMBIE dataset filename
    obs_filename = cmct_dir + '/data/IMBIE/imbie_greenland_2021_Gt.csv'

    # Other regions --- these are used for Antarctica only, so set them to None for Greenland
    obs_east_filename = None
    obs_west_filename = None
    obs_peninsula_filename = None
    
elif icesheet== "AIS":
    projection = "EPSG:3031"  
    
    # Template for the model filenames
    mod_filename_template = '/home/jovyan/shared-public/CmCt/models/ISMIP6/lithk_AIS_*_*_hist_std.nc' 
    
    # Shapefile for basin partitioning
    shape_filename = cmct_dir + '/data/IMBIE/ANT_Basins_IMBIE2_v1.6/ANT_Basins_IMBIE2_v1.6.shp'
    
    # IMBIE dataset filename
    obs_filename = cmct_dir + '/data/IMBIE/imbie_antarctica_2021_Gt.csv'
    
    # Other regions --- these are used for Antarctica and are not yet published, so set them to None for now
    obs_east_filename = None
    obs_west_filename = None
    obs_peninsula_filename= None

else:
    raise ValueError("Invalid icesheet value. Must be 'Greenland' or 'Antarctica'.")


In [3]:
# Check if  observation file exists
if not os.path.exists(obs_filename):
    raise FileNotFoundError(f"Observation file not found: {obs_filename}")

if icesheet== "AIS":
    if (obs_east_filename and os.path.exists(obs_east_filename)) and \
           (obs_west_filename and os.path.exists(obs_west_filename)) and \
           (obs_peninsula_filename and os.path.exists(obs_peninsula_filename)):
        # Check if regional observation files exist 
        if not os.path.exists(obs_east_filename):
            raise FileNotFoundError(f"Observation file not found: {obs_east_filename}")
        if not os.path.exists(obs_west_filename):
            raise FileNotFoundError(f"Observation file not found: {obs_west_filename}")
        if not os.path.exists(obs_peninsula_filename):
            raise FileNotFoundError(f"Observation file not found: {obs_peninsula_filename}")


# Mass change comparision processing

In [4]:
# Get the list of all model data files
nc_filenames = glob.glob(mod_filename_template)

# Loop through each file 
for nc_filename in nc_filenames:
    print(f"\nProcessing:{nc_filename}")
    
    # Open model file
    mod_ds = xr.open_dataset(nc_filename,use_cftime=True)
    time_var = mod_ds['time']
    
    calendar_type = time_var.to_index().calendar
    start_date_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end_date_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d')
    
    # Adjust day to be 30 ( to avoid error if it's the 31st day in a 360_day calendar)
    start_date_cftime = cftime.datetime(start_date_dt.year, start_date_dt.month, min(start_date_dt.day, 30), calendar=calendar_type)    
    
    end_date_cftime = cftime.datetime(end_date_dt.year, end_date_dt.month, min(end_date_dt.day, 30), calendar=calendar_type)  
    
    start_date_fract = start_date_cftime.year + (start_date_cftime.dayofyr-1) / days_in_year(start_date_cftime)
    end_date_fract = end_date_cftime.year + (end_date_cftime.dayofyr-1) / days_in_year(end_date_cftime)
    
    
    # Read IMBIE mass change from start to end date
    IMBIE_mass_change = process_imbie_data(obs_filename, start_date_fract, end_date_fract, mass_balance_column)
    
    # Calculate time-varying modeled mass change
    model_mass_change = process_model_data(mod_ds,time_var, IMBIE_mass_change, start_date_cftime, end_date_cftime, start_date_fract, end_date_fract, rho_ice, projection, shape_filename, icesheet)
    
    # Calculate IMBIE-model mass change residuals
    imbie_model_residuals = calculate_model_imbie_residuals(start_date_fract, end_date_fract, icesheet, model_mass_change, IMBIE_mass_change, mass_balance_column, obs_east_filename, obs_west_filename, obs_peninsula_filename)

    
    # Extract the base name of the nc file (without .nc extension)
    nc_base_filename = os.path.basename(nc_filename).replace('.nc', '')
    
    # Create the CSV filename by combining the output path and the base nc filename with .csv extension
    csv_filename = os.path.join(output_path, f"{nc_base_filename}.csv")

    
    # Write  the mass change comparison  to csv file  
    write_mass_change_comparison_all_dates(icesheet, model_mass_change, imbie_model_residuals, mass_balance_type, start_date_fract, end_date_fract, csv_filename)
    


Processing:/home/jovyan/shared-public/CmCt/models/ISMIP6/lithk_AIS_AWI_PISM1_hist_std.nc
The selected dates {start_date_cftime} and {end_date_cftime} are within the range of the model data.

Writing data to CSV file: /home/jovyan/CmCt/notebooks/IMBIE/lithk_AIS_AWI_PISM1_hist_std.csv

Processing:/home/jovyan/shared-public/CmCt/models/ISMIP6/lithk_AIS_IMAU_IMAUICE1_hist_std.nc
The selected dates {start_date_cftime} and {end_date_cftime} are within the range of the model data.

Writing data to CSV file: /home/jovyan/CmCt/notebooks/IMBIE/lithk_AIS_IMAU_IMAUICE1_hist_std.csv

Processing:/home/jovyan/shared-public/CmCt/models/ISMIP6/lithk_AIS_UCIJPL_ISSM_hist_std.nc
The selected dates {start_date_cftime} and {end_date_cftime} are within the range of the model data.

Writing data to CSV file: /home/jovyan/CmCt/notebooks/IMBIE/lithk_AIS_UCIJPL_ISSM_hist_std.csv
